In [0]:
!wget https://raw.githubusercontent.com/avinashsai/Cross-domain-sentiment-analysis/master/Dataset/Actualdata/Books/Bookstrain.txt
!wget https://raw.githubusercontent.com/avinashsai/Cross-domain-sentiment-analysis/master/Dataset/Actualdata/Books/Bookstest.txt
  
!wget https://raw.githubusercontent.com/avinashsai/Cross-domain-sentiment-analysis/master/Dataset/Actualdata/Dvd/Dvdtrain.txt
!wget https://raw.githubusercontent.com/avinashsai/Cross-domain-sentiment-analysis/master/Dataset/Actualdata/Dvd/Dvdtest.txt
  
!wget https://raw.githubusercontent.com/avinashsai/Cross-domain-sentiment-analysis/master/Dataset/Actualdata/Electronics/Electronicstrain.txt
!wget https://raw.githubusercontent.com/avinashsai/Cross-domain-sentiment-analysis/master/Dataset/Actualdata/Electronics/Electronicstest.txt
  
!wget https://raw.githubusercontent.com/avinashsai/Cross-domain-sentiment-analysis/master/Dataset/Actualdata/Kitchen/Kitchentrain.txt
!wget https://raw.githubusercontent.com/avinashsai/Cross-domain-sentiment-analysis/master/Dataset/Actualdata/Kitchen/Kitchentest.txt

In [0]:
embedding_index = {}
with open('glove.840B.300d.txt','r',encoding='utf-8') as f:
  for line in f.readlines():
    ws = line.split(' ')
    word = ws[0]
    vector = np.asarray(ws[1:],'float32')
    embedding_index[word] = vector

In [0]:
import os
import re
import sys
import pickle
import time
import copy
from copy import deepcopy
import numpy as np
import pandas as pd
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim.models import doc2vec,fasttext,FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
import tensorflow as tf
import keras
from keras.layers import *
from keras.models import Model,Sequential
from keras.callbacks import *
from keras.metrics import *
import keras.backend as K
from keras.engine import Layer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
def preprocess(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.lower().strip()

In [0]:
Xtrain = []
with open('Electronicstrain.txt','r',encoding='latin1') as f:
    for line in f.readlines():
        Xtrain.append(preprocess(line[:-1]))

ytrain = np.zeros(1600)
ytrain[0:800] = 1
Xtrain,ytrain = shuffle(Xtrain,ytrain,random_state=0)
Xtrain,Xval,ytrain,yval = train_test_split(Xtrain,ytrain,test_size=0.2,random_state=42)

In [0]:
Xtest = []
with open('Electronicstest.txt','r',encoding='latin1') as f:
    for line in f.readlines():
        Xtest.append(preprocess(line[:-1]))

ytest = np.zeros(400)
ytest[0:200] = 1
Xtest,ytest = shuffle(Xtest,ytest,random_state=0)

In [0]:
train_l = len(Xtrain)
val_l = len(Xval)
test_l = len(Xtest)

In [0]:
words = []
for sentence in Xtrain:
    words.append(sentence.split())

In [0]:
maxsenlen = 60
batchsize = 32
numepochs = 30

## FastText + LSTM

In [0]:
mincount = 1
vecsize = 100
winsize = 5
totalepoch = 20

In [0]:
np.random.seed(0)
ft = FastText(min_count=mincount,size=vecsize,window=winsize)
ft.build_vocab(sentences=words)
ft.train(sentences=words,total_examples=len(words),epochs=totalepoch)

In [0]:
def load_vectors(sentence):
    senlen = len(sentence.split())
    maxlen = min(maxsenlen,senlen)
    vec = [ft.wv[sentence[i]] for i  in range(maxlen) if sentence[i] in ft.wv.vocab]
    if(len(vec)<maxsenlen):
        vec+=[np.zeros(vecsize) for i in range(len(vec),maxsenlen)]
    return np.asarray(vec)

In [0]:
train_vectors = np.zeros((train_l,maxsenlen,vecsize))
for i in range(train_l):
    train_vectors[i,:,:] = load_vectors(Xtrain[i])

In [0]:
val_vectors = np.zeros((val_l,maxsenlen,vecsize))
for i in range(val_l):
    val_vectors[i,:,:] = load_vectors(Xval[i])

In [0]:
test_vectors = np.zeros((test_l,maxsenlen,vecsize))
for i in range(test_l):
    test_vectors[i,:,:] = load_vectors(Xtest[i])

In [0]:
def model_lstm():
    inp = Input(shape=(maxsenlen,vecsize))
    lstm1 = LSTM(100,recurrent_dropout=0.2,return_sequences=True)(inp)
    lstm1 = LSTM(50,return_sequences=False)(lstm1)
    out = Dense(1,activation='sigmoid')(lstm1)
    
    m1 = Model(inputs=inp,outputs=out)
    return m1

In [70]:
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model1 = model_lstm()
    if(run==0):
        print(model1.summary())
    model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model1.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model1.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 60, 100)           0         
_________________________________________________________________
lstm_32 (LSTM)               (None, 60, 100)           80400     
_________________________________________________________________
lstm_33 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 51        
Total params: 110,651
Trainable params: 110,651
Non-trainable params: 0
_________________________________________________________________
None
F1 Score 0.5507108208795483 
Accuracy Score 0.5439999999999999 
Avg_time  192.57592730522157


## PV(DBOW)+FNN

In [0]:
tagged_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(words)]

In [0]:
np.random.seed(0)
pv_dbow = Doc2Vec(vector_size=vecsize, window=2, min_count=1, workers=4)
pv_dbow.build_vocab(tagged_documents)
pv_dbow.train(tagged_documents,total_examples=pv_dbow.corpus_count,epochs=20)

In [0]:
train_vectors = np.zeros((train_l,vecsize))
for i in range(train_l):
  train_vectors[i] = pv_dbow.infer_vector(Xtrain[i].split())

In [0]:
val_vectors = np.zeros((val_l,vecsize))
for i in range(val_l):
  val_vectors[i] = pv_dbow.infer_vector(Xval[i].split())

In [0]:
test_vectors = np.zeros((test_l,vecsize))
for i in range(test_l):
  test_vectors[i] = pv_dbow.infer_vector(Xtest[i].split())

In [0]:
def model_dense():
    inp = Input(shape=(vecsize,))
    dense1 = Dense(50,activation='relu')(inp)
    dense1 = Dropout(0.5)(dense1)
    dense1 = Dense(16,activation='relu')(dense1)
    dense1 = Dropout(0.5)(dense1)
    out = Dense(1,activation='sigmoid')(dense1)
    m1 = Model(inputs=inp,outputs=out)
    return m1

In [77]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model2 = model_dense()
    if(run==0):
        print(model2.summary())
    model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model2.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model2.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_71 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_41 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_72 (Dense)             (None, 16)                816       
_________________________________________________________________
dropout_42 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 17        
Total params: 5,883
Trainable params: 5,883
Non-trainable params: 0
_________________________________________________________________
None
F1 

### PV(DM)+FNN

In [0]:
np.random.seed(0)
pv_dm = Doc2Vec(dm=1,vector_size=vecsize, window=2, min_count=1, workers=4)
pv_dm.build_vocab(tagged_documents)
pv_dm.train(tagged_documents,total_examples=pv_dm.corpus_count,epochs=20)

In [0]:
train_vectors = np.zeros((train_l,vecsize))
for i in range(train_l):
  train_vectors[i] = pv_dm.infer_vector(Xtrain[i].split())

In [0]:
val_vectors = np.zeros((val_l,vecsize))
for i in range(val_l):
  val_vectors[i] = pv_dm.infer_vector(Xval[i].split())

In [0]:
test_vectors = np.zeros((test_l,vecsize))
for i in range(test_l):
  test_vectors[i] = pv_dm.infer_vector(Xtest[i].split())

In [82]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model2 = model_dense()
    if(run==0):
        print(model2.summary())
    model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model2.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model2.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_49 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_86 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_51 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_87 (Dense)             (None, 16)                816       
_________________________________________________________________
dropout_52 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_88 (Dense)             (None, 1)                 17        
Total params: 5,883
Trainable params: 5,883
Non-trainable params: 0
_________________________________________________________________
None
F1 

In [0]:
maxsenlen = 60
embeddim = 300

In [0]:
tok = Tokenizer()
tok.fit_on_texts(Xtrain)

word_index = tok.word_index

train = tok.texts_to_sequences(Xtrain)
train_vectors = pad_sequences(train,maxsenlen)

val = tok.texts_to_sequences(Xval)
val_vectors = pad_sequences(val,maxsenlen)

test = tok.texts_to_sequences(Xtest)
test_vectors = pad_sequences(test,maxsenlen)

In [0]:
embedding_matrix = np.zeros((len(word_index)+1,embeddim))
for word, i in word_index.items():
  embedding_vector = embedding_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

### LSTM

In [0]:
def lstm_model():
  inp = Input(shape=(maxsenlen,))
  
  embed = Embedding(len(word_index)+1,embeddim,weights=[embedding_matrix],trainable=False)(inp)
  
  lstm_out = LSTM(100,recurrent_dropout=0.2,return_sequences=True)(embed)
  lstm_out = LSTM(50,return_sequences=False)(lstm_out)
  
  out = Dense(16,activation='relu')(lstm_out)
  out = Dropout(0.5)(out)
  out = Dense(1,activation='sigmoid')(out)
  
  m2 = Model(inputs=inp,outputs=[out])
  return m2

In [46]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model3 = lstm_model()
    if(run==0):
        print(model3.summary())
    model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model3.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model3.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 60)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 60, 300)           5461200   
_________________________________________________________________
lstm_12 (LSTM)               (None, 60, 100)           160400    
_________________________________________________________________
lstm_13 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dense_36 (Dense)             (None, 16)                816       
_________________________________________________________________
dropout_21 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 1)                 17        
Total para

## Bi-LSTM

In [0]:
def bilstm_model():
  inp = Input(shape=(maxsenlen,))
  
  embed = Embedding(len(word_index)+1,embeddim,weights=[embedding_matrix],trainable=False)(inp)
  
  lstm_out = Bidirectional(LSTM(100,recurrent_dropout=0.2,return_sequences=True))(embed)
  lstm_out = Bidirectional(LSTM(50,return_sequences=False))(lstm_out)
  
  out = Dense(16,activation='relu')(lstm_out)
  out = Dropout(0.5)(out)
  out = Dense(1,activation='sigmoid')(out)
  
  m3 = Model(inputs=inp,outputs=[out])
  return m3

In [48]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model4 = bilstm_model()
    if(run==0):
        print(model4.summary())
    model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model4.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model4.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 60)                0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 60, 300)           5461200   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 200)           320800    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100)               100400    
_________________________________________________________________
dense_46 (Dense)             (None, 16)                1616      
_________________________________________________________________
dropout_26 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 17        
Total para

In [0]:
num_filters = 100
filter_sizes = [3,4,5]
drop = 0.5

## Random CNN

In [0]:
def random_cnn():
  inp = Input(shape=(maxsenlen,), dtype='int32')
  embedding = Embedding(input_dim=len(word_index)+1, output_dim=embeddim, input_length=maxsenlen)(inp)
  reshape = Reshape((maxsenlen,embeddim,1))(embedding)

  conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

  maxpool_0 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
  maxpool_1 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
  maxpool_2 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

  concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
  flatten = Flatten()(concatenated_tensor)
  dropout = Dropout(drop)(flatten)
  output = Dense(units=1, activation='sigmoid')(dropout)
  
  m4 = Model(inputs=inp,outputs=output)
  return m4

In [55]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model5 = random_cnn()
    if(run==0):
        print(model5.summary())
    model5.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model5.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model5.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 60, 300)      5461200     input_29[0][0]                   
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 60, 300, 1)   0           embedding_13[0][0]               
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 58, 1, 100)   90100       reshape_2[0][0]                  
__________________________________________________________________________________________________
conv2d_5 (

### Static CNN

In [0]:
def static_cnn():
  inp = Input(shape=(maxsenlen,), dtype='int32')
  embedding = Embedding(input_dim=len(word_index)+1, output_dim=embeddim, weights=[embedding_matrix],trainable=False)(inp)
  reshape = Reshape((maxsenlen,embedddim,1))(embedding)

  conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

  maxpool_0 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
  maxpool_1 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
  maxpool_2 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

  concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
  flatten = Flatten()(concatenated_tensor)
  dropout = Dropout(drop)(flatten)
  output = Dense(units=1, activation='sigmoid')(dropout)
  
  m5 = Model(inputs=inp,outputs=output)
  return m5

In [57]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model6 = random_cnn()
    if(run==0):
        print(model6.summary())
    model6.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model6.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model6.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 60, 300)      5461200     input_34[0][0]                   
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 60, 300, 1)   0           embedding_18[0][0]               
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 58, 1, 100)   90100       reshape_7[0][0]                  
__________________________________________________________________________________________________
conv2d_20 